# Đồ án cuối kỳ

Thông tin nhóm:
    
Phạm Quang Phước Nguyên - MSSV: 1612441

Nguyễn Cao Nhân - MSSV: 1612451

---
## Đặt vấn đề

---
## Mô tả dữ liệu

---
## Import

In [1]:
#khai báo thư viện
import numpy as np
%matplotlib inline
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

---
## Đọc dữ liệu từ file csv vào dataframe

---
## Trực quan hóa dữ liệu

---
## Tiền xử lí dữ liệu

---
## Train mô hình

---
## Test mô hình

---
## Đánh giá kết quả

---
## Kết luận